In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import itertools
import keras
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from keras_preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras import optimizers
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
import math
import datetime
import time

In [ ]:

img_width, img_height = 224, 224

#Create a bottleneck file
top_model_weights_path = 'bottleneck_fc_model.h5'
# loading up our datasets
train_data_dir='/content/drive/MyDrive/data_rename/train'
validation_data_dir='/content/drive/MyDrive/data_rename/test'
# number of epochs to train top model
epochs = 7 #this has been changed after multiple model run
# batch size used by flow_from_directory and predict_generator
batch_size = 50

In [ ]:
#Loading vgc16 model
vgg16=applications.VGG16(include_top=False, weights='imagenet')
datagen = ImageDataGenerator(rescale=1. / 255)
#needed to create the bottleneck .npy files

58889256/58889256 [==============================] - 0s 0us/step


In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
#train- creation of weights and features using vgg16
start = datetime.datetime.now()

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_train_samples = len(generator.filenames)
num_classes = len(generator.class_indices)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))

bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train)

np.save('bottleneck_features_train.npy', bottleneck_features_train)
end= datetime.datetime.now()
elapsed= end-start
print('Time: ', elapsed)

Found 3360 images belonging to 4 classes.


<ipython-input-8-c6ab365cfe62>:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_train = vgg16.predict_generator(generator, predict_size_train)


Time:  0:29:25.919700


In [ ]:
#validation -creation of weights and features using vgg16
start = datetime.datetime.now()

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)

nb_validation_samples = len(generator.filenames)
num_classes = len(generator.class_indices)

predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))

bottleneck_features_validation = vgg16.predict_generator(generator, predict_size_validation)

np.save('bottleneck_features_validation.npy', bottleneck_features_validation)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Found 450 images belonging to 4 classes.


<ipython-input-9-1f7c383a8521>:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  bottleneck_features_validation = vgg16.predict_generator(generator, predict_size_validation)


Time:  0:04:24.799303


In [ ]:
start = datetime.datetime.now()
datagen_top = ImageDataGenerator(rescale=1./255)
generator_top = datagen_top.flow_from_directory(
         train_data_dir,
         target_size=(img_width, img_height),
         batch_size=batch_size,
         class_mode='categorical',
         shuffle=False)

nb_train_samples = len(generator_top.filenames)
num_classes = len(generator_top.class_indices)

# load the bottleneck features saved earlier
train_data = np.load('bottleneck_features_train.npy')

# get the class lebels for the training data, in the original order
train_labels = generator_top.classes

# convert the training labels to categorical vectors
train_labels = to_categorical(train_labels, num_classes=num_classes)
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)


Found 3360 images belonging to 4 classes.
Time:  0:00:01.020797


In [ ]:
generator_top = datagen_top.flow_from_directory(
         validation_data_dir,
         target_size=(img_width, img_height),
         batch_size=batch_size,
         class_mode=None,
         shuffle=False)

nb_validation_samples = len(generator_top.filenames)

validation_data = np.load('bottleneck_features_validation.npy')


validation_labels = generator_top.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)


Found 450 images belonging to 4 classes.


In [ ]:
start = datetime.datetime.now()
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels,
      epochs=5,
      batch_size=batch_size,
      validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)

(eval_loss, eval_accuracy) = model.evaluate(
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)


Epoch 1/5
68/68 [==============================] - 9s 119ms/step - loss: 2.0043 - accuracy: 0.4089 - val_loss: 1.2295 - val_accuracy: 0.3756
Epoch 2/5
68/68 [==============================] - 5s 78ms/step - loss: 1.0953 - accuracy: 0.5110 - val_loss: 1.0319 - val_accuracy: 0.5667
Epoch 3/5
68/68 [==============================] - 6s 94ms/step - loss: 0.9915 - accuracy: 0.5702 - val_loss: 0.9246 - val_accuracy: 0.5600
Epoch 4/5
68/68 [==============================] - 7s 105ms/step - loss: 0.9171 - accuracy: 0.6080 - val_loss: 1.2768 - val_accuracy: 0.3422
Epoch 5/5
9/9 [==============================] - 0s 18ms/step - loss: 0.9871 - accuracy: 0.5578
[INFO] accuracy: 55.78%
[INFO] Loss: 0.9870734810829163
Time:  0:00:33.977915


In [ ]:
start = datetime.datetime.now()
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit(train_data, train_labels,
      epochs=7,
      batch_size=batch_size,
      validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)

(eval_loss, eval_accuracy) = model.evaluate(
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)


Epoch 1/7


/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


68/68 [==============================] - 3s 41ms/step - loss: 1.3496 - acc: 0.3426 - val_loss: 1.2791 - val_acc: 0.4111
Epoch 2/7
68/68 [==============================] - 3s 39ms/step - loss: 1.2365 - acc: 0.4378 - val_loss: 1.2076 - val_acc: 0.5000
Epoch 3/7
68/68 [==============================] - 3s 38ms/step - loss: 1.1682 - acc: 0.4786 - val_loss: 1.1188 - val_acc: 0.5289
Epoch 4/7
68/68 [==============================] - 3s 39ms/step - loss: 1.1162 - acc: 0.5170 - val_loss: 1.0810 - val_acc: 0.5267
Epoch 5/7
68/68 [==============================] - 3s 38ms/step - loss: 1.0756 - acc: 0.5271 - val_loss: 1.0410 - val_acc: 0.5933
Epoch 6/7
68/68 [==============================] - 4s 53ms/step - loss: 1.0097 - acc: 0.5839 - val_loss: 1.0222 - val_acc: 0.5356
Epoch 7/7
9/9 [==============================] - 0s 12ms/step - loss: 0.9797 - acc: 0.5511
[INFO] accuracy: 55.11%
[INFO] Loss: 0.9796505570411682
Time:  0:00:41.874724


In [ ]:
#method3
import keras
from keras import optimizers
start = datetime.datetime.now()
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(100, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.5))
model.add(Dense(50, activation=keras.layers.LeakyReLU(alpha=0.3)))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

history = model.fit(train_data, train_labels,
      epochs=7,
      batch_size=batch_size,
      validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)

(eval_loss, eval_accuracy) = model.evaluate(
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Epoch 1/7
68/68 [==============================] - 3s 42ms/step - loss: 1.2875 - acc: 0.4176 - val_loss: 1.2389 - val_acc: 0.4244
Epoch 2/7
68/68 [==============================] - 3s 39ms/step - loss: 1.0385 - acc: 0.5524 - val_loss: 1.0813 - val_acc: 0.5067
Epoch 3/7
68/68 [==============================] - 3s 39ms/step - loss: 0.9370 - acc: 0.5973 - val_loss: 1.0000 - val_acc: 0.5644
Epoch 4/7
68/68 [==============================] - 4s 54ms/step - loss: 0.8607 - acc: 0.6280 - val_loss: 0.9985 - val_acc: 0.5644
Epoch 5/7
68/68 [==============================] - 4s 54ms/step - loss: 0.8079 - acc: 0.6685 - val_loss: 0.8854 - val_acc: 0.5889
Epoch 6/7
68/68 [==============================] - 3s 40ms/step - loss: 0.7880 - acc: 0.6687 - val_loss: 0.8970 - val_acc: 0.6156
Epoch 7/7
9/9 [==============================] - 0s 13ms/step - loss: 0.9049 - acc: 0.6089
[INFO] accuracy: 60.89%
[INFO] Loss: 0.9048681855201721
Time:  0:00:21.805951


In [ ]:
#method4
start = datetime.datetime.now()
model = Sequential()
model.add(Flatten(input_shape=train_data.shape[1:]))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.6))
model.add(Dense(50, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop',
          loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_data, train_labels,
      epochs=20,
      batch_size=batch_size,
      validation_data=(validation_data, validation_labels))

model.save_weights(top_model_weights_path)

(eval_loss, eval_accuracy) = model.evaluate(
 validation_data, validation_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
print("[INFO] Loss: {}".format(eval_loss))
end= datetime.datetime.now()
elapsed= end-start
print ('Time: ', elapsed)

Epoch 1/20
68/68 [==============================] - 6s 79ms/step - loss: 1.5940 - accuracy: 0.2789 - val_loss: 1.3549 - val_accuracy: 0.3356
Epoch 2/20
68/68 [==============================] - 5s 68ms/step - loss: 1.3882 - accuracy: 0.2887 - val_loss: 1.3396 - val_accuracy: 0.2533
Epoch 3/20
68/68 [==============================] - 5s 69ms/step - loss: 1.3532 - accuracy: 0.3423 - val_loss: 1.2881 - val_accuracy: 0.3200
Epoch 4/20
68/68 [==============================] - 5s 77ms/step - loss: 1.2978 - accuracy: 0.3988 - val_loss: 1.1986 - val_accuracy: 0.5289
Epoch 5/20
68/68 [==============================] - 6s 82ms/step - loss: 1.2299 - accuracy: 0.4455 - val_loss: 1.1619 - val_accuracy: 0.5489
Epoch 6/20
68/68 [==============================] - 5s 78ms/step - loss: 1.1870 - accuracy: 0.4685 - val_loss: 1.3857 - val_accuracy: 0.3267
Epoch 7/20
68/68 [==============================] - 5s 70ms/step - loss: 1.1587 - accuracy: 0.4771 - val_loss: 1.0372 - val_accuracy: 0.5844
Epoch 8/20
68